# Imports & Setup

In [7]:
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd
import os

# Define where we will save the database on disk
# We use the 'data/processed' folder we created earlier
PERSIST_DIRECTORY = "../data/processed/chroma_db"

# Setup the embedding function (The Translator)
# We use the same model as before: 'all-MiniLM-L6-v2'
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

print("Setup complete. Database path:", PERSIST_DIRECTORY)

Setup complete. Database path: ../data/processed/chroma_db


# Initialize Client & Collection

In [8]:
# Initialize the ChromaDB client (Persistent means it saves to disk)
client = chromadb.PersistentClient(path=PERSIST_DIRECTORY)

# Create (or get if it already exists) a collection
collection = client.get_or_create_collection(
    name="customer_feedback",
    embedding_function=embedding_func,  # Chroma will handle vectorization automatically!
    metadata={"hnsw:space": "cosine"},  # Use Cosine Similarity
)

print(f"Collection '{collection.name}' is ready.")

Collection 'customer_feedback' is ready.


# Adding Data (Ingestion)

In [19]:
import pandas as pd
from pathlib import Path
import sys

# --- CONFIGURATION ---
# Professional Path Handling using pathlib
PROJECT_ROOT = Path("..")
RAW_DATA_DIR = PROJECT_ROOT / "data" / "raw"

print(f"📂 Scanning directory: {RAW_DATA_DIR.resolve()}")

# 1. Auto-Discovery Logic
# Instead of hardcoding a filename, we find any .csv file in the folder.
# This makes the pipeline robust to filename changes.
csv_files = list(RAW_DATA_DIR.glob("*.csv"))

if not csv_files:
    print("\n❌ ERROR: No CSV file found in 'data/raw/'")
    print("👉 ACTION REQUIRED: Download the CSV manually and place it in that folder.")
    # Stop execution gracefully
    raise FileNotFoundError("Missing raw data file.")

target_file = csv_files[0]
print(f"✅ Found file: {target_file.name}")

# 2. Load Data
try:
    df = pd.read_csv(target_file)
    print(f"📊 Loaded {len(df)} rows.")
except Exception as e:
    print(f"❌ Error reading CSV: {e}")
    raise e

# 3. Data Cleaning Pipeline
# Standardize column headers (snake_case)
df.columns = (
    df.columns.str.lower().str.strip().str.replace(" ", "_").str.replace("-", "_")
)

# Dynamic Text Column Detection
# The code looks for common names for the complaint column
possible_cols = ["customer_complaint", "consumer_complaint_narrative", "complaint"]
text_col = next((c for c in possible_cols if c in df.columns), None)

if not text_col:
    raise ValueError(
        f"Could not find a complaint text column. Available columns: {df.columns.tolist()}"
    )

print(f"🎯 Using text column: '{text_col}'")

# Remove empty complaints
df = df.dropna(subset=[text_col])

# OPTIONAL: Sampling for Local Development
# If the dataset is huge, we sample 1000 rows to keep the demo snappy on CPU.
# In production, you would remove this limit.
if len(df) > 1000:
    df_subset = df.sample(1000, random_state=42).copy()
else:
    df_subset = df.copy()

# 4. Prepare Embeddings
documents = df_subset[text_col].tolist()

metadatas = []
for index, row in df_subset.iterrows():
    # Defensive programming: Handle missing metadata gracefully
    status = str(row.get("status", "Unknown"))
    ticket = str(row.get("ticket_#", row.get("ticket_id", "Unknown")))

    metadatas.append(
        {"status": status, "ticket_id": ticket, "source": "Manual Download"}
    )

# ... (Keep the previous data loading and cleaning logic unchanged) ...

ids = [f"ticket_{i}" for i in range(len(documents))]

print(f"📝 Prepared {len(documents)} documents for ingestion.")

# --- DATABASE RESET & INGESTION LOGIC ---

# 1. Check current database state
current_count = collection.count()
print(f"🔍 Current Vector Store Status: {current_count} documents.")

# 2. Reset: Purge old test data (the 7 dummy records) if they exist
if current_count > 0:
    print(
        "🧹 Detecting legacy/test data. Purging collection to ensure a clean slate..."
    )

    # Retrieve all existing IDs to delete them
    existing_data = collection.get()
    existing_ids = existing_data["ids"]

    if existing_ids:
        collection.delete(ids=existing_ids)

    print(f"✅ Collection cleared. Current count: {collection.count()}")

# 3. Ingest Real Data
print(f"🚀 Ingesting {len(documents)} real Comcast documents into ChromaDB...")
print("   (This process calculates embeddings and might take 1-2 minutes on CPU...)")

collection.add(documents=documents, ids=ids, metadatas=metadatas)

# 4. Final Verification
final_count = collection.count()
print("-" * 50)
print(f"🎉 SUCCESS! Vector Store successfully populated.")
print(f"📊 Total Real Documents: {final_count}")
print("-" * 50)

📂 Scanning directory: /home/guga/Work/rag-feedback-analyzer/data/raw
✅ Found file: comcast_fcc_complaints_2015.csv
📊 Loaded 2225 rows.
🎯 Using text column: 'customer_complaint'
📝 Prepared 1000 documents for ingestion.
🔍 Current Vector Store Status: 7 documents.
🧹 Detecting legacy/test data. Purging collection to ensure a clean slate...
✅ Collection cleared. Current count: 0
🚀 Ingesting 1000 real Comcast documents into ChromaDB...
   (This process calculates embeddings and might take 1-2 minutes on CPU...)
--------------------------------------------------
🎉 SUCCESS! Vector Store successfully populated.
📊 Total Real Documents: 1000
--------------------------------------------------


# Semantic Search (The Query)

In [22]:
# --- SEMANTIC SEARCH VALIDATION ---

# Now that we have real data, let's test the "Brain" of the system.
# We will search for a concept, not necessarily exact keywords.

# 1. Define a query relevant to the Telecom domain
query_text = "internet speed is extremely slow and connection drops"

print(f"🔎 Searching for: '{query_text}'\n")

# 2. Query the Vector Database
results = collection.query(
    query_texts=[query_text],
    n_results=3,  # Retrieve top 3 most similar complaints
)

# 3. Display Results
for i, doc in enumerate(results["documents"][0]):
    meta = results["metadatas"][0][i]
    distance = results["distances"][0][i] if "distances" in results else "N/A"

    print(f"📄 Result {i + 1}:")
    print(f"   Ticket ID: {meta['ticket_id']}")
    print(f"   Status:    {meta['status']}")
    print(f"   Complaint: {doc[:200]}...")  # Show first 200 chars to keep it clean
    print("-" * 50)

🔎 Searching for: 'internet speed is extremely slow and connection drops'

📄 Result 1:
   Ticket ID: 360349
   Status:    Pending
   Complaint: Slow Internet Speeds...
--------------------------------------------------
📄 Result 2:
   Ticket ID: 364377
   Status:    Open
   Complaint: Slow Internet speeds...
--------------------------------------------------
📄 Result 3:
   Ticket ID: 224912
   Status:    Closed
   Complaint: Slow internet speeds...
--------------------------------------------------
